In [6]:
import oceanliner

In [ ]:
oceanliner.

In [ ]:
def get_survey_track_test(ds, sampling_details):
     
    """Calculates the survey indices and track based on the sampling details for the dataset for all days.


    Args:
        ds (xarray.core.dataset.Dataset): MITgcm LLC4320 data for all days
        sampling_details (dict): It includes number of days, waypoints, and depth range, horizontal and vertical platform speed. These can typical (default) or user-specified, in the                                      case where user specfies only some of the details the default values will be used for rest.

    Returns:
        survey_track (xarray.core.dataset.Dataset): Returns the track (lat, lon, depth, time) of the sampling trajectory based on the type of sampling                               
        survey_indices (xarray.core.dataset.Dataset): Returns the indices (i, j, k, time) of the sampling trajectory based on the type of sampling
        sampling_details (dict): Returns the modified sampling_details by filling in the missing parameters with defaults.
        
    Raises: 
        Sampling strategy is invalid: If a sampling strategy is not specified or different from the available strategies - sim_utcd, sim_glider, sim_mooring, wave_glider, sail_drone
    

    """
    
    
    survey_time_total = (ds.time.values.max() - ds.time.values.min()) # (timedelta) - limits the survey to a total time
    survey_end_time = ds.time.isel(time=0).data + survey_time_total # end time of survey
    # Convert lon, lat and z to index i, j and k with f_x, f_y and f_z
    # XC, YC and Z are the same at all times, so select a single time
    X = ds.XC.isel(time=0) 
    Y = ds.YC.isel(time=0)
    i = ds.i
    j = ds.j
    z = ds.Z.isel(time=0)
    k = ds.k
    f_x = interpolate.interp1d(X[0,:].values, i)
    f_y = interpolate.interp1d(Y[:,0].values, j)
    f_z = interpolate.interp1d(z, k, bounds_error=False)

    # Get boundaries and center of model region
    model_boundary_n = Y.max().values
    model_boundary_s = Y.min().values
    model_boundary_w = X.min().values
    model_boundary_e = X.max().values
    model_xav = ds.XC.isel(time=0, j=0).mean(dim='i').values
    model_yav = ds.YC.isel(time=0, i=0).mean(dim='j').values
    
    
    set_defaults(sampling_details)
   
    # --------- define sampling -------
    SAMPLING_STRATEGY = sampling_details['SAMPLING_STRATEGY']

    # ----- define x/y/z/t points to interpolate to
    # for moorings, location is fixed so a set of waypoints is not needed.
    # however, for "sim_mooring", tile/repeat the sampling x/y/t to form 2-d arrays,
    # so the glider/uCTD interpolation framework can be used.
    # - and for "mooring", skip the step of interpolating to "points" and interpolate directly to the new x/y/t/z 
    if SAMPLING_STRATEGY == 'sim_mooring':
        # time sampling is one per model timestep
#         ts = ds.time.values / 24 # convert from hours to days
        ts = ds.time.values # in hours
        n_samples = ts.size
        n_profiles = n_samples
        # same sampling for T/S/U/V for now. NOTE: change this later!        
        zs = np.tile(sampling_details['zmooring_TS'], int(n_samples)) # sample depths * # of samples 
        xs = sampling_details['xmooring'] * np.ones(np.size(zs))  # all samples @ same x location
        ys = sampling_details['ymooring'] * np.ones(np.size(zs))  # all samples @ same y location
        ts = np.repeat(ts, len(sampling_details['zmooring_TS']))  # tile to match size of other fields. use REPEAT, not TILE to get the interpolation right.

#         # depth sampling - different for TS and UV
#         zs_TS = np.tile(zmooring_TS, int(n_samples))
#         zs_UV = np.tile(zmooring_UV, int(n_samples))
#         xs_TS = xmooring * np.ones(np.size(zs_TS))
#         xs_UV = xmooring * np.ones(np.size(zs_UV))
#         ys_TS = ymooring * np.ones(np.size(zs_TS))
#         ys_UV = ymooring * np.ones(np.size(zs_UV))
#         ts_TS = np.tile(ts, int(n_samples))
        
        
#         lon_TS = xr.DataArray(xs_TS,dims='points'),
#         lat_TS = xr.DataArray(ys_TS,dims='points'),
#         dep_TS = xr.DataArray(zs_TS,dims='points'),
#         time_TS = xr.DataArray(ts,dims='points')
               
#         lon = lon_TS
#         lat = lat_TS
#         dep = dep_TS
#         time = time_TS
    elif SAMPLING_STRATEGY == 'mooring':
        ts = ds.time.values # in hours
        # same sampling for T/S/U/V for now. NOTE: change this later!  
        zs = sampling_details['zmooring_TS'] 
        xs = sampling_details['xmooring']
        ys = sampling_details['ymooring']
    else:
        # --- if not a mooring, define waypoints  
    
        # define x & y waypoints and z range
        # xwaypoints & ywaypoints must have the same size
        if sampling_details['PATTERN'] == 'lawnmower':
            # "mow the lawn" pattern - define all waypoints
            if not(SAMPLING_STRATEGY == 'trajectory_file'):
                # generalize the survey for this region
                xwaypoints = model_boundary_w + 1 + [0, 0, 0.5, 0.5, 1, 1, 1.5, 1.5, 2, 2]
                ywaypoints = model_boundary_s + [1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2]
        elif sampling_details['PATTERN'] == 'back-forth':
            if not(SAMPLING_STRATEGY == 'trajectory_file'):
                # repeated back & forth transects - define the end-points
                xwaypoints = model_xav + [-1, 1]
                ywaypoints = model_yav + [-1, 1]
            # repeat waypoints based on total # of transects: 
            dkm_per_transect = great_circle(xwaypoints[0], ywaypoints[0], xwaypoints[1], ywaypoints[1]) # distance of one transect in km
#           # time per transect, seconds, as a np.timedelta64 value
            t_per_transect = np.timedelta64(int(dkm_per_transect * 1000 / sampling_details['hspeed']), 's')    
            num_transects = np.round(survey_time_total / t_per_transect)
            for n in np.arange(num_transects):
                xwaypoints = np.append(xwaypoints, xwaypoints[-2])
                ywaypoints = np.append(ywaypoints, ywaypoints[-2])
        if SAMPLING_STRATEGY == 'trajectory_file':
            xwaypoints = sampling_details['xwaypoints']
            ywaypoints = sampling_details['ywaypoints']
        # if the survey pattern repeats, add the first waypoint to the end of the list of waypoints:
        if sampling_details['AT_END'] == 'repeat': 
            xwaypoints = np.append(xwaypoints, xwaypoints[0])
            ywaypoints = np.append(ywaypoints, ywaypoints[0])                
        
####### Different function
        # vertical resolution
        #if ((SAMPLING_STRATEGY != 'wave_glider') and (SAMPLING_STRATEGY != 'sail_drone')): 
        zresolution = 600 # meters
        
        # max depth can't be deeper than the max model depth in this region
        sampling_details['zrange'][1] = -np.min([-sampling_details['zrange'][1], ds.Depth.isel(time=1).max(...).values])        
        
        zprofile = np.arange(sampling_details['zrange'][0],sampling_details['zrange'][1],-zresolution) # depths for one profile
        ztwoway = np.append(zprofile,zprofile[-1::-1])
        # time resolution of sampling (dt):
        if ((SAMPLING_STRATEGY != 'wave_glider') and (SAMPLING_STRATEGY != 'sail_drone')):
            vspeed = 0 
            dt=0
            # for each timestep dt 
            deltah = sampling_details['hspeed']*dt # horizontal distance traveled per sample
            deltav = sampling_details['vspeed']*dt # vertical distance traveled per sample
        else:
            dt = zresolution / sampling_details['vspeed'] # sampling resolution in seconds
            dt_td64 = np.timedelta64(int(dt), 's') # np.timedelta64 format
            # for each timestep dt 
            deltah = sampling_details['hspeed']*dt # horizontal distance traveled per sample
            deltav = sampling_details['vspeed']*dt # vertical distance traveled per sample
            # determine the sampling locations in 2-d space
            # initialize sample locations xs, ys, zs, ts
            xs = []
            ys = []
            zs = []
            ts = []
            dkm_total = 0 
            
            for w in np.arange(len(xwaypoints)-1):
                # interpolate between this and the following waypoint:
                dkm = great_circle(xwaypoints[w], ywaypoints[w], xwaypoints[w+1], ywaypoints[w+1])
                # number of time steps (vertical measurements) between this and the next waypoint
                nstep = int(dkm*1000 / deltah) 
                yi = np.linspace(ywaypoints[w], ywaypoints[w+1], nstep)
                xi = np.linspace(xwaypoints[w], xwaypoints[w+1], nstep)
                xi = xi[0:-1] # remove last point, which is the next waypoint
                xs = np.append(xs, xi) # append
                yi = yi[0:-1] # remove last point, which is the next waypoint
                ys = np.append(ys, yi) # append
                dkm_total = dkm_total + dkm           
                # cumulative survey time to this point, in seconds, as a np.timedelta64 value
                t_total = np.timedelta64(int(dkm_total * 1000 / sampling_details['hspeed']), 's')
    

        
            
            # cut off the survey after survey_time_total
        if t_total > survey_time_total:
            break 
                
        # km for one lap of the survey
        dkm_once = dkm_total 

        # if time is less than survey_time_total, trigger AT_END behavior:
        if t_total < survey_time_total:
            if sampling_details['AT_END'] == 'repeat': 
                # start at the beginning again
                # - determine how many times the survey repeats:
                num_transects = np.round(survey_time_total / t_total)
                x_once = xs
                y_once = ys
                for n in np.arange(num_transects):
                    xs = np.append(xs, x_once)
                    ys = np.append(ys, y_once)
                    dkm_total += dkm_once
            elif sampling_details['AT_END'] == 'reverse': 
                # turn around & go in the opposite direction
                # - determine how many times the survey repeats:
                num_transects = np.round(survey_time_total / t_total)
                x_once = xs
                y_once = ys
                # append both a backward & another forward transect
                for n in np.arange(np.ceil(num_transects/2)):
                    xs = np.append(np.append(xs, x_once[-2:1:-1]), x_once)
                    ys = np.append(np.append(ys, y_once[-2:1:-1]), y_once)
                    dkm_total += dkm_once*2


        # repeat (tile) the two-way sampling depths 
        # - number of profiles we make during the survey:
        n_profiles = np.ceil(xs.size / ztwoway.size)
        zs = np.tile(ztwoway, int(n_profiles))
        zs = zs[0:xs.size] # limit to # of sample times
        ts = ds.time.isel(time=0).data + dt_td64 * np.arange(xs.size)
        # get rid of points with sample time > survey_time_total
        if survey_time_total.astype('float64')> 0:
            idx = np.argmin(np.abs(ts - survey_end_time))# index of ts closest to survey_end_time
            print('originally, ', idx, ' points')
            # make sure this is multiple of the # of profiles:
            idx = int(np.floor((idx+1)/len(ztwoway)) * (len(ztwoway)))
            xs = xs[:idx]
            ys = ys[:idx]
            ts = ts[:idx]
            zs = zs[:idx]
            n_profiles = np.ceil(xs.size / ztwoway.size)
            # update t_total
            t_total = np.diff(ts[[0,-1]])
            t_total_seconds = int(t_total)/1e9 # convert from nanoseconds to seconds
            # use the speed to determine dkm_total (time * hspeed)
            dkm_total = t_total_seconds * sampling_details['hspeed'] / 1000
            print('limited to ', idx, 'points: n_profiles=', n_profiles, ', ', len(zprofile), 'depths per profile, ', len(ztwoway), 'depths per two-way')
            
        sampling_details['distance_total_km'] = dkm_total
        sampling_details['time_total_s'] = t_total_seconds  
        # -- end if not a mooring
        
    # ----- Assemble dataset: -----
    # (same regardless of sampling strategy - EXCEPT "mooring")
    if not SAMPLING_STRATEGY == 'mooring':
        # - real (lat/lon) coordinates:
        survey_track = xr.Dataset(
            dict(
                lon = xr.DataArray(xs,dims='points'),
                lat = xr.DataArray(ys,dims='points'),
                dep = xr.DataArray(zs,dims='points'),
                time = xr.DataArray(ts,dims='points'),
                n_profiles = n_profiles
            )
        )
        # - transform to i,j,k coordinates:
        survey_indices= xr.Dataset(
            dict(
                i = xr.DataArray(f_x(survey_track.lon), dims='points'),
                j = xr.DataArray(f_y(survey_track.lat), dims='points'),
                k = xr.DataArray(f_z(survey_track.dep), dims='points'),
                time = xr.DataArray(survey_track.time, dims='points'),
            )
        )
    elif SAMPLING_STRATEGY == 'mooring':
        survey_track = xr.Dataset(
            dict(
                lon = xr.DataArray(xs*[1], dims='position'),
                lat = xr.DataArray(ys*[1], dims='position'),
                dep = xr.DataArray(zs, dims='depth'),
                time = xr.DataArray(ts, dims='time')

            )
        )
        # - transform to i,j,k coordinates:
        survey_indices= xr.Dataset(
            dict(
                i = xr.DataArray(f_x(survey_track.lon), dims='position'),
                j = xr.DataArray(f_y(survey_track.lat), dims='position'),
                k = xr.DataArray(f_z(survey_track.dep), dims='depth'),
                time = xr.DataArray(survey_track.time, dims='time'),
            )
        )
    # store SAMPLING_STRATEGY and DERIVED_VARIABLES in survey_track so they can be used later
    survey_track['SAMPLING_STRATEGY'] = SAMPLING_STRATEGY
#     survey_track['DERIVED_VARIABLES'] = sampling_details['DERIVED_VARIABLES']
#    survey_track['SAVE_PRELIMINARY'] = sampling_details['SAVE_PRELIMINARY']
    return survey_track, survey_indices, sampling_details